### Pre-defined Method

In [1]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import re

def test_compute_metrics(label, pred):
    MAE = []

    for i in range(5):
        if pred[i][-1] == '.':
            pred[i] = pred[i][:-1]
        
        MAE.append(1 - mean_absolute_error([float(label[i])], [float(pred[i])]))
    
    return MAE

In [2]:
import pandas as pd

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv",
                     index_col=0)

### Zero-shot LLM Inference

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
logging.disable(logging.INFO) 
logging.disable(logging.WARNING)

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv")  # 데이터 불러오기 (First Impression V2 데이터의 Test set 불러오기)

MODEL_DIR = "/home/visi2/juphome/PersonalityAI/Instruction_tuning/Bllossom-8B/"  # 모델 이름
ORIGIN_MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.num_parameters())  # 모델 사이즈 출력; KoSOLAR : 10,804,924,416 (10.8B)

tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token
#streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

genconfig = GenerationConfig(  # Decoding/Sampling Strategy
    max_new_tokens=50, # 최대 생성 토큰, 1024
    do_sample=True, 
    min_new_tokens = 15,
    temperature=0.9,
    repetition_penalty=1.3,
    top_k=50,
    top_p=0.92,
    eos_token_id=model.config.eos_token_id
)

result_df = pd.DataFrame(columns=["Text", "Inference"])  # 결과를 저장할 데이터프레임

for content in tqdm(FI_test['transcription'].to_list()):
    user = content
    inputs = tokenizer.encode(zero_shot2 + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # 입력 데이터 토크나이징
    outputs = model.generate(inputs, 
                             generation_config = genconfig)

    # Post-Processing
    out_text = tokenizer.decode([el.item() for el in outputs[0]])  # output 숫자 값 -> 텍스트로 바꾸기
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")  # output에서 input 빼기

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})  # 만들어진 출력 저장하기
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
import pandas as pd

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['transcription'][error_idx].to_list()):
        user = data
     #   conversation = [{'role': 'system', 'content': zero_shot2}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(zero_shot2 + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)

In [ ]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae

--------------------

### One-shot LLM Inference (Kernel Restart Required for GPU Memory)

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
import pandas as pd
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv")

MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
ORIGIN_MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.config.eos_token_id)
print(model.num_parameters())  # KoSOLAR : 10,804,924,416 (10.8B)
tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

one_shot_text = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

ko_prompt_one_shot = zero_shot2 + '문장: "' + one_shot_text + '",\n 결과- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'

genconfig = GenerationConfig(
    max_new_tokens=50, 
    do_sample=True, 
    min_new_tokens = 15,
    temperature=0.9,
    repetition_penalty=1.3,
 #   no_repeat_ngram_size=3,
    top_k=50,
    top_p=0.92,
    eos_token_id=model.config.eos_token_id
)

result_df = pd.DataFrame(columns=["Text", "Inference"])

cnt = 0

for content in tqdm(FI_test['transcription'].to_list()):
    user = content
    inputs = tokenizer.encode(ko_prompt_one_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  


    outputs = model.generate(inputs, 
                             generation_config = genconfig)
    out_text = tokenizer.decode([el.item() for el in outputs[0]])
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
import pandas as pd

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['transcription'][error_idx].to_list()):
        user = data
       # conversation = [{'role': 'system', 'content': en_prompt}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(ko_prompt_one_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)

In [ ]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae

------------

### five-shot LLM Inference (Kernel Restart Required for GPU Memory)

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
import pandas as pd
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv")

MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
ORIGIN_MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.config.eos_token_id)
print(model.num_parameters())  # KoSOLAR : 10,804,924,416 (10.8B)

tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token
#streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

few_shot_text0 = "네 그럼 14위는 어떤 분으로 하는 게 좋을까요?"
few_shot_text1 = "편의점 귀엽다"
few_shot_text2 = "술 팔까요? 아 근데"
few_shot_text3 = "진짜요? 그러면 해주실 수 있어요."
few_shot_text4 = "그렇죠 그렇죠 그렇죠 여기 여기 메인 무대가 무대랑 좌석인 거죠?"

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

ko_prompt_few_shot = zero_shot2 + '문장: "' + few_shot_text0 + '",\n 결과- [O: 0.9762 , C: 0.9792 , E: 0.8333 , A: 0.6458 , N: 0.2083 ] \n' + \
                    '문장: "' + few_shot_text1 + '",\n 결과- [O: 0.6667 , C: 0.7292, E: 0.8542, A: 0.5625, N: 0.2708] \n' + \
                    '문장: "' + few_shot_text2 + '",\n 결과- [O: 0.5625, C: 0.8333, E: 0.8125, A: 0.3125, N: 0.2708 ] \n' + \
                    '문장: "' + few_shot_text3 + '",\n 결과- [O: 0.7917, C: 0.5625, E: 0.8542, A: 0.7083, N: 0.3750] \n' + \
                    '문장: "' + few_shot_text4 + '",\n 결과- [O: 0.9375, C: 0.9583, E: 0.8750, A: 0.3542, N: 0.1875 ] \n'


genconfig = GenerationConfig(
    max_new_tokens=50, 
    do_sample=True, 
    min_new_tokens = 15,
    temperature=0.9,
    repetition_penalty=1.3,
 #   no_repeat_ngram_size=3,
    top_k=50,
    top_p=0.92,
    eos_token_id=model.config.eos_token_id
)

result_df = pd.DataFrame(columns=["Text", "Inference"])

cnt = 0

for content in tqdm(FI_test['transcription'].to_list()):
    user = content
    inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda") 

    outputs = model.generate(inputs, 
                #      streamer=streamer,
                             generation_config = genconfig)
    
    out_text = tokenizer.decode([el.item() for el in outputs[0]])
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
import pandas as pd

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['transcription'][error_idx].to_list()):
        user = data
     #   conversation = [{'role': 'system', 'content': en_prompt}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)

In [ ]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae

### ETC

In [ ]:
p1 = "너는 지금부터 문장에 대해 BIG 5 성격 검사의 결과를 보여주는 전문 계산기야. 너는 전문가로써 BIG 5에 대한 정보를 충분히 알고있고 \
주어진 문장에 대해 5가지 요소를 가지는 BIG 5 성격 검사로 개성을 평가해야만 해. \
이 때, 문장이 주어지면 각각의 요소(개방성, 성실성, 우호성, 외향성, 신경증)에 대해 0부터 1사이의 값을 가지도록 숫자로 평가해야하고, \
모든 요소에 대해 최대한 세밀하고 정확하게 값을 출력해야 해. \
결과를 출력할 때, BIG 5에 대한 설명이나 이유에 대해서는 필요 없이 숫자 값만 보여줘. <요소 - 숫자> 형식으로 출력해.\
문장 : 오늘 워크샵은 정말 기대돼!>"

p2 = "당신은 문장에 대해 BIG 5 성격 검사의 결과를 보여주는 전문가 입니다. 주어진 문장을 읽고, 다음과 같은 OCEAN 요소들을 각각 0에서 1 사이의 값으로 나타내세요. O(Openness): 개방성, C(Conscientiousness): 성실성, \
E(Extraversion): 외향성, A(Agreeableness): 친화성, N(Neuroticism): 신경성, 문장을 이용하여 각 요소의 점수를 계산할 때, 다음과 같은 접근을 사용할 수 있습니다: \
Openness (O): 문장에 포함된 창의성, 호기심, 복잡성에 대한 언급 등을 고려합니다. Conscientiousness (C): 문장에서 나타나는 계획성, 철저함, 조직성을 고려합니다. \
Extraversion (E): 문장에서 외향적인 특성, 사회적 활동에 대한 언급을 고려합니다. Agreeableness (A): 문장에서 타인에 대한 관심, 협동성, 친화력을 고려합니다. \
Neuroticism (N): 문장에 포함된 감정적 불안, 긴장, 우울 등을 고려합니다. \
이러한 기준을 이용하여 주어진 문장을 분석하여 각 요소에 대한 0에서 1 사이의 값을 계산해보세요. \
계산된 값은 다음의 예시처럼 다섯 개의 요소에 대한 배열로 표현되어야 합니다. \
예시) 다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분히 못 자서 가위를 어딘가로 옮겼을 수도 있지만 맹세컨대 - [0.8222, 0.6699, 0.4860, 0.6458, 0.6813]"

In [ ]:
base_prompt = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'
one_shot_text = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."

zero = base_prompt + '문장: "' + "<EXAMPLE SENTENCE OF FI_TEST DATASET>" + '",\n 결과- [O: '

one = base_prompt + '문장: "' + one_shot_text + '",\n 결과- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n' + '문장: "' + "<EXAMPLE SENTENCE OF FI_TEST DATASET>" + '",\n 결과- [O: '

print("Zero shot Input: ", "\n", zero )
print("\n\n" ,"========================="*15, "\n\n")
print("One shot Input: ", "\n", one)

### prompt example mining

In [ ]:
import pandas as pd

val = pd.read_csv("/data/visi2/FI/KETI_test_from_ver0.4.csv",
                 index_col=0)

In [ ]:
val.describe().T

In [ ]:
open_high = val[val['openness'] == val['openness'].max()]
open_low = val.sort_values(by='openness')
cons_high = val[val['conscientiousness'] == val['conscientiousness'].max()]
cons_low = val[val['conscientiousness'] == val['conscientiousness'].min()]
extra_high = val[val['extraversion'] == val['extraversion'].max()]
extra_low = val.sort_values(by='extraversion')
agree_high = val.sort_values(by='agreeableness',ascending=False)
agree_low = val.sort_values(by='agreeableness')
neuro_high = val[val['neuroticism'] == val['neuroticism'].max()]
neuro_low = val.sort_values(by='neuroticism')

In [ ]:
neuro_low

In [ ]:
res = {
    open_high['transcription'].to_list()[0] : open_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    open_low['transcription'].to_list()[1] : open_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    cons_high['transcription'].to_list()[0] : cons_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    cons_low['transcription'].to_list()[0] : cons_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    extra_high['transcription'].to_list()[0] : extra_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    extra_low['transcription'].to_list()[1] : extra_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    agree_high['transcription'].to_list()[1] : agree_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    agree_low['transcription'].to_list()[1] : agree_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    neuro_high['transcription'].to_list()[0] : neuro_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    neuro_low['transcription'].to_list()[1] : neuro_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
}

len(res), res

### Ten-Shot Inference

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
import pandas as pd
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv")

MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
ORIGIN_MODEL_DIR = "MLP-KTLim/llama-3-Korean-Bllossom-8B"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.config.eos_token_id)
print(model.num_parameters())  # KoSOLAR : 10,804,924,416 (10.8B)

tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

few_shot_text0 = "고민 중인데 노라조, \n"
few_shot_text1 = "네 괜찮은 것 같아요. 그러면 4위 \n"
few_shot_text2 = "10분에 70만 원이요 거의 뭐 \n"
few_shot_text3 = "맞아 잔치국수 어 잔치국수 괜찮다 \n"
few_shot_text4 = "되게 빡센데 1박 2일이면 많은 걸 보여줘야 되는데 그것도 괜찮은데 \n"
few_shot_text5 = "쇼핑도 해야돼 저도 P에요 \n"
few_shot_text6 = "할 수 있는데 캐리커처는 약간 또 다른 재능의 영역이라고 생각해요. 그것도 약간 교육이 필요한 것 같은데 \n"
few_shot_text7 = "저는 우선 그래프를 봤을 때 연령대가 나오는데 일단 중년층이랑 고등학생과 이제 대학생 비율이 높다는 점을 고려를 해서 \
동서남북 지도를 살펴봤을 때 저는 남쪽이랑 동쪽이 좀 유리하다라고 생각이 들었고 그중에서도 동쪽이 가장 유리하지 않나라는 생각이 들었어요 \
우선 동쪽에는 나이 드신 분들이라 진학 상담 부스가 있어서 고등학생들이 많이 몰리게 되는데 딱 방문자 수 비율을 봤을 때 \
이제 중년 46세부터 55세 분들과 이제 16세부터 25세의 비율이 높다 보니까 좀 동쪽을 하면은 좀 유동인구 연령층과도 잘 맞아가지고 좋지 않을까라는 생각이 들었어요. \n"
few_shot_text8 = "강아지 키우세요? 귀여워요 어떡해 \n"
few_shot_text9 = "그래서 또 그럼 둘째 날 언제 가지? 점심에 가요? \n"

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

ko_prompt_few_shot = zero_shot2 + '문장: "' + few_shot_text0 + '",\n 결과- [O: 0.6875, C: 0.4792, E: 0.4375, A: 0.7708, N: 0.4375] \n' + \
                    '문장: "' + few_shot_text1 + '",\n 결과- [O: 0.6667, C: 0.7292, E: 0.8542, A: 0.5625, N: 0.2708 ] \n' + \
                    '문장: "' + few_shot_text2 + '",\n 결과- [O: 0.5208, C: 0.6875, E: 0.7920, A: 0.6458, N: 0.2500 ] \n' + \
                    '문장: "' + few_shot_text3 + '",\n 결과- [O: 0.4583, C: 0.6875, E: 0.7083, A: 0.5833, N: 0.3333] \n' + \
                    '문장: "' + few_shot_text4 + '",\n 결과- [O: 0.5000, C: 0.5625, E: 0.6875, A: 0.5625, N: 0.2708 ] \n' + \
                    '문장: "' + few_shot_text5 + '",\n 결과- [O: 0.5833, C: 0.4583, E: 0.8125, A: 0.5420, N: 0.4583] \n' + \
                    '문장: "' + few_shot_text6 + '",\n 결과- [O: 0.7292, C: 0.4792, E: 0.5000, A: 0.4583, N: 0.7292 ] \n' + \
                    '문장: "' + few_shot_text7 + '",\n 결과- [O: 0.9792, C: 0.9792, E: 0.8333, A: 0.6458, N: 0.2083] \n' + \
                    '문장: "' + few_shot_text8 + '",\n 결과- [O: 0.7920, C: 0.5625, E: 0.8542, A: 0.7083, N: 0.3750] \n' + \
                    '문장: "' + few_shot_text9 + '",\n 결과- [O: 0.5625, C: 0.8333, E: 0.8125, A: 0.3125, N: 0.2708] \n'

genconfig = GenerationConfig(
    max_new_tokens=50, 
    do_sample=True, 
    min_new_tokens = 15,
    temperature=0.9,
    repetition_penalty=1.3,
    top_k=50,
    top_p=0.92,
    eos_token_id=model.config.eos_token_id
)

result_df = pd.DataFrame(columns=["Text", "Inference"])

cnt = 0

for content in tqdm(FI_test['transcription'].to_list()):
    user = content
    inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda") 

    outputs = model.generate(inputs, 
                             generation_config = genconfig)
    
    out_text = tokenizer.decode([el.item() for el in outputs[0]])
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

In [ ]:
result_df['Inference'].to_list()

In [ ]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [ ]:
score

In [ ]:
import pandas as pd

FI_test = pd.read_csv("/data/visi2/Dataset/KETI_val_from_ver0.4.csv",
                     index_col=0)
FI_test['label'] = FI_test['OCEAN'].map(lambda x : [float(i) for i in x[1:-1].split(',')])

In [ ]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [ ]:
error_idx

In [ ]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['transcription'][error_idx].to_list()):
        user = data
     #   conversation = [{'role': 'system', 'content': en_prompt}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)


In [ ]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae